In [1]:
import pandas as pd
import sagemaker
from sklearn.model_selection import train_test_split
import numpy as np
import boto3
from sagemaker.estimator import Estimator
# from sagemaker.amazon.amazon_estimator import get_image_uri

In [2]:
from sagemaker import image_uris

In [3]:
df=pd.read_csv('salary.csv')
df.head()

,YearsExperience,Salary
0,1.1,39343
1,1.3,46205
2,1.5,37731
3,2.0,43525
4,2.2,39891


In [4]:
X,y=df['YearsExperience'],df['Salary']

In [5]:
X=np.array(X).astype('float32')
y=np.array(y).astype('float32')

In [6]:
X,y=X.reshape(35,1),y.reshape(35,1)

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [8]:
container=image_uris.retrieve('tensorflow',
                             boto3.Session().region_name,
                              version='2.1',
                              py_version='py3',
                              instance_type='ml.t3.medium',
                          
                              image_scope='training')

In [51]:
linear=Estimator(linearimageuri,sagemaker.get_execution_role(),1,instance_type='ml.c5.2xlarge',sagemaker_session=sagemaker.Session())

In [54]:
linear.set_hyperparameters(feature_dim=1,
                          predictor_type='regressor',
                          mini_batch_size=5,
                          epochs=3,
                          num_models=10,
                          loss='squared_loss')

In [33]:
linearimageuri='382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest'

In [36]:
import io
import sagemaker.amazon.common as smac

In [47]:
buf=io.BytesIO()
smac.write_numpy_to_dense_tensor(buf,X_train,y_train.reshape(-1))
buf.seek(0)

0

In [48]:
boto3.resource('s3').Bucket('salaryestimator').Object(os.path.join('traindata','linearlearner')).upload_fileobj(buf)

In [44]:
import os

In [55]:
linear.fit({'train':'s3://salaryestimator/traindata/linearlearner'})

2020-10-15 03:01:05 Starting - Starting the training job...
2020-10-15 03:01:08 Starting - Launching requested ML instances......
2020-10-15 03:02:31 Starting - Preparing the instances for training.........
2020-10-15 03:03:54 Downloading - Downloading input data
2020-10-15 03:03:54 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/15/2020 03:04:12 INFO 140026608404288] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'in